## Step 2

In [31]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.aljazeera.com/where/mozambique/'
r1 = requests.get(url)
soup1 = BeautifulSoup(r1.content, 'lxml')
news = soup1.find_all('h3')

In [3]:
n = 10
link_list = []
title_list = []
content_list = []

for i in np.arange(0, n):
    url = news[i].find('a')['href']
    link = 'https://www.aljazeera.com'+url
    link_list.append(link)
    
    title = news[i].find('span').get_text()
    title_list.append(title)
    
    article = requests.get(link)
    soup_article = BeautifulSoup(article.content, 'lxml')
    body = soup_article.find_all('p')
    subhead = soup_article.find_all('p',class_ = "article__subhead css-1wt8oh6")
    footer = soup_article.find_all('p',class_ = "site-footer__social-section-title css-0")
    if len(subhead) != 0:
        subhead = subhead[0].get_text()
    if len(footer) != 0:
        footer = footer[0].get_text()
    paragraph = []
    for i in np.arange(0, len(body)):
        p = body[i].get_text()
        if p != subhead and p != footer:
            if len(p)>0:
                paragraph.append(p)

    content = " ".join(paragraph).replace('\n','')
    content_list.append(content)


In [4]:
for i in np.arange(0,n):
    filename = title_list[i] + '.json'
    with open(filename, 'w') as outfile:
        outfile.write(content_list[i])

## Step 3

In [26]:
from spacy_langdetect import LanguageDetector
import spacy
from spacy.language import Language

In [27]:
def create_lang_detector(nlp, name):
    return LanguageDetector()

Language.factory("language_detector", func=create_lang_detector)
nlp = spacy.load('en_core_web_sm')  # 1
nlp.add_pipe('language_detector', last=True)
for i in tqdm(np.arange(0,n)):
    text_content = content_list[i]
    doc = nlp(text_content) #3
    detect_language = doc._.language #4
    print(detect_language)

 10%|█         | 1/10 [00:00<00:07,  1.24it/s]

{'language': 'en', 'score': 0.9999966195479728}
{'language': 'en', 'score': 0.9999971657223669}
{'language': 'en', 'score': 0.9999969205862639}


 60%|██████    | 6/10 [00:01<00:01,  2.26it/s]

{'language': 'en', 'score': 0.9999967372198298}
{'language': 'en', 'score': 0.9999974396977409}
{'language': 'en', 'score': 0.999998307249834}
{'language': 'en', 'score': 0.9999956732065018}


100%|██████████| 10/10 [00:01<00:00,  6.04it/s]

{'language': 'en', 'score': 0.9999977702191877}
{'language': 'en', 'score': 0.9999974738642898}
{'language': 'en', 'score': 0.999996305575282}


## Step 4

In [29]:
from textblob import TextBlob

In [35]:
sentiment=[]
for content,title in zip(content_list,title_list):
    txt = TextBlob(content)
    p = txt.sentiment.polarity
    s = txt.sentiment.subjectivity
    sentiment.append([title,p,s])
    
df_sa = pd.DataFrame(sentiment, columns =['News', 'Polarity', 'Subjectivity'])
df_sa

,News,Polarity,Subjectivity
0,Floods hit South Africa’s KwaZu­lu-Na­tal prov...,0.062153,0.453472
1,Mozam­bique: Cy­clone Gombe death toll ris­es ...,-0.130263,0.276316
2,Mozam­bique an­nounces new prime min­is­ter af...,0.096266,0.454221
3,Analy­sis: Can African gas re­place Russ­ian s...,0.068602,0.369939
4,Dozens dead from Trop­i­cal Storm Ana in south...,0.043182,0.395455
5,South­ern Africa bloc SADC ex­tends Mozam­biqu...,0.109444,0.328333
6,Cli­mate change and famine | Start Here,-0.500000,0.550000
7,"In Mozam­bique, Kagame says Rwan­dan troops’ w...",0.033076,0.310387
8,"Rwan­da, Mozam­bique forces re­cap­ture port c...",0.103472,0.312963
9,"Rwan­da de­ploys 1,000 sol­diers to Mozam­biqu...",-0.006414,0.353620


## Step 5

In [49]:
from nltk import FreqDist
import plotly.express as px

In [52]:
stopwords = nltk.corpus.stopwords.words('english')

word_freq

{'heavy': 2,
 'rains': 2,
 'south': 2,
 'africa': 1,
 'forced': 1,
 'hundreds': 2,
 'people': 4,
 'flee': 1,
 'homes': 2,
 'east': 1,
 'coast,': 1,
 'according': 1,
 'officials,': 1,
 'month': 1,
 'worst': 1,
 'floods': 3,
 'living': 1,
 'memory': 1,
 'killed': 1,
 '400.the': 1,
 'weekend': 1,
 'hit': 1,
 'infrastructure,': 1,
 'roads,': 1,
 'bridges': 1,
 'damaged': 1,
 'buildings,': 1,
 'mainly': 1,
 'port': 1,
 'city': 1,
 'durban,': 1,
 'capital': 1,
 'kwazulu-natal': 4,
 'province': 1,
 'home': 1,
 '3.9': 1,
 'million': 1,
 'people.': 1,
 '“some': 1,
 'areas': 1,
 'inaccessible': 1,
 'become': 1,
 'islands': 1,
 'stage,”': 1,
 'premier': 1,
 'sihle': 1,
 'zikalala': 2,
 'said': 4,
 'news': 2,
 'conference': 1,
 'sunday.no': 1,
 'fatalities': 1,
 'immediately': 1,
 'recorded,': 1,
 'added:': 1,
 '“we': 1,
 'yet': 1,
 'receive': 1,
 'full': 1,
 'comprehensive': 1,
 'report': 1,
 'impact': 1,
 'latest': 1,
 'rains,': 1,
 'reports': 1,
 'come': 1,
 'worrying': 1,
 'picture': 1,
 'howe

In [71]:
for content,title in zip(content_list,title_list):
    word_list = []
    for word in content.split():
        word_lower = word.lower()
        if word not in stopwords:
            word_list.append(word.lower())

    word_freq = dict(FreqDist(word_list))
    fig = px.bar(pd.DataFrame(word_freq, index=["Count"]).T.sort_values(by=["Count"],ascending=False), y="Count",title = title)
    fig.show()